In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [3]:
train_data = pd.read_csv("train.csv") # 학습용 데이터
test_data = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
submission_id = test_data['id']

In [5]:
import pandas as pd

def fill_strategic_ver_columns(df):
    df['id_strategic_ver'] = df.apply(
        lambda x: 1 if x['business_unit'] == 'ID' and pd.isnull(x['id_strategic_ver']) else x['id_strategic_ver'],
        axis=1
    )
    df['it_strategic_ver'] = df.apply(
        lambda x: 1 if x['business_unit'] == 'IT' and pd.isnull(x['it_strategic_ver']) else x['it_strategic_ver'],
        axis=1
    )
    df['id_strategic_ver'].fillna(0, inplace=True)
    df['it_strategic_ver'].fillna(0, inplace=True)
    return df

# 결측치 채우는 로직 적용
train_data = fill_strategic_ver_columns(train_data)
test_data = fill_strategic_ver_columns(test_data)

# train_data에 apply 수행
train_data['idit_strategic_ver'] = train_data['business_unit'].apply(lambda x: 1 if x in ['ID', 'IT'] else 0)

# test_data에 apply 수행
test_data['idit_strategic_ver'] = test_data['business_unit'].apply(lambda x: 1 if x in ['ID', 'IT'] else 0)

# customer_type 컬럼에서 같은 의미인 값을 변경
train_data['customer_type'] = train_data['customer_type'].replace({
    'End-Customer': 'End Customer',
    'Specifier/ Influencer': 'Specifier / Influencer',
    'Homeowner': 'Home Owner'
})

test_data['customer_type'] = test_data['customer_type'].replace({
    'End-Customer': 'End Customer',
    'Specifier/ Influencer': 'Specifier / Influencer',
    'Homeowner': 'Home Owner'
})




# 결측치 비율 확인
missing_values = train_data.isnull().mean() * 100

print(missing_values)


bant_submit                 0.000000
expected_budget            59.321405
customer_country            1.656014
business_unit               0.000000
com_reg_ver_win_rate       75.432975
customer_idx                0.000000
lead_date                   0.000000
customer_type              74.134471
customer_history            0.000000
enterprise                  0.000000
historical_existing_cnt    76.802307
id_strategic_ver            0.000000
it_strategic_ver            0.000000
idit_strategic_ver          0.000000
customer_job               31.590752
lead_from_channel           5.556586
lead_description           50.933405
lead_desc_length            0.000000
event_name                  1.908970
inquiry_type                1.586873
prefer_ver_count           68.942141
prefer_ver_mean            68.942141
product_category           32.671715
product_subcategory        84.426382
product_modelname          84.436500
customer_country.1          1.656014
customer_position           0.000000
r

In [6]:
# 고유 값과 그 비율 계산
unique_values = train_data['ver_win_rate_mean_upper'].unique()
value_counts = train_data['ver_win_rate_mean_upper'].value_counts(normalize=True) * 100

# 결과 출력
print("Unique Values:")
print(unique_values)
print("\nValue Counts:")
print(value_counts)


Unique Values:
[ 1.  0. nan]

Value Counts:
ver_win_rate_mean_upper
0.0    79.247434
1.0    20.752566
Name: proportion, dtype: float64


In [7]:
#expected_budget 전처리
train_data['expected_budget'] = train_data['expected_budget'].str.replace('_', ' ', regex=False)

# Replacing '-' with '~'
train_data['expected_budget'] = train_data['expected_budget'].str.replace('-', '~', regex=False)


#expected_budget 전처리
test_data['expected_budget'] = test_data['expected_budget'].str.replace('_', ' ', regex=False)

# Replacing '-' with '~'
test_data['expected_budget'] = test_data['expected_budget'].str.replace('-', '~', regex=False)


def remove_duplicates(text, delimiter=';'):
    if pd.isna(text):  # Checks if text is NaN
        return text
    unique_elements = set(text.split(delimiter))
    return delimiter.join(sorted(unique_elements))



# Applying the function to the 'expected_budget' column
train_data['expected_budget'] = train_data['expected_budget'].apply(lambda x: remove_duplicates(x, ';'))

# Applying the function to the 'expected_budget' column
test_data['expected_budget'] = test_data['expected_budget'].apply(lambda x: remove_duplicates(x, ';'))

train_data['expected_budget'] = train_data['expected_budget'].str.lower()

test_data['expected_budget'] = test_data['expected_budget'].str.lower()

In [8]:
# Splitting 'event_name' by '_' and expanding the results into new columns
event_name_splits = train_data['event_name'].str.split('_', expand=True)

# Creating new column names based on the number of splits
new_column_names = [f'event_split_{i}' for i in range(event_name_splits.shape[1])]

# Adding these new columns to the original DataFrame
train_data[new_column_names] = event_name_splits




# Splitting 'event_name' by '_' and expanding the results into new columns
event_name_splits = test_data['event_name'].str.split('_', expand=True)

# Creating new column names based on the number of splits
new_column_names = [f'event_split_{i}' for i in range(event_name_splits.shape[1])]

# Adding these new columns to the original DataFrame
test_data[new_column_names] = event_name_splits





# 출력 설정 조정
pd.set_option('display.max_rows', None)  # 행 생략 없이 전부 보여줌
pd.set_option('display.max_columns', None)  # 열 생략 없이 전부 보여줌
pd.set_option('display.width', None)  # 콘솔 너비 제한 없애기
pd.set_option('display.max_colwidth', None)  # 컬럼 너비 제한 없애기


In [9]:
# 고유 값과 그 비율 계산
unique_values = train_data['customer_type'].unique()
value_counts = train_data['customer_type'].value_counts(normalize=True) * 100

# 결과 출력
print("Unique Values:")
print(unique_values)
print("\nValue Counts:")
print(value_counts)


Unique Values:
['End Customer' nan 'Solution Eco-Partner' 'Specifier / Influencer'
 'Service Partner' 'Channel Partner' 'Etc.' 'Software/Solution Provider'
 'Dealer/Distributor' 'Installer/Contractor' 'Other' 'End-user'
 'Developer' 'Technician' 'Corporate' 'Engineer' 'Others' 'Consultant'
 'Manager / Director' 'Home Owner' 'Interior Designer' 'HVAC Engineer'
 'Reseller' 'Installer' 'Architect/Consultant' 'Distributor'
 'System Integrator' 'Administrator' 'Commercial end-user'
 'Technical Assistant' 'Software / Solution Provider']

Value Counts:
customer_type
End Customer                    69.389751
Specifier / Influencer          16.742730
Channel Partner                  8.919025
Service Partner                  2.275394
Solution Eco-Partner             0.951884
Installer/Contractor             0.339027
Corporate                        0.202112
HVAC Engineer                    0.149954
Engineer                         0.130395
Developer                        0.110836
Technician    

In [10]:
# 범주형 변수 식별
categorical_vars = [
    'expected_budget', 'bant_submit', 'customer_country', 'business_unit', 'customer_idx', 'customer_type', 
    'enterprise', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job',
    'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 
    'customer_country.1', 'customer_position', 'response_corporate', 'expected_timeline', 
    'ver_cus', 'ver_pro', 'business_area', 'business_subarea', 'lead_owner', 
    'Detailed_Address', 'City', 'Country', 'is_converted', 'customer_history', 'transfer_agreement', 'lead_description',
    'event_split_1', 'event_split_2', 'event_split_3', 'event_split_4', 'event_split_5',
    'event_split_6', 'event_split_7', 'event_split_8', 'event_split_9', 'event_split_10'
]

# 수치형 변수 식별
numerical_vars =  ['bant_submit',
  'com_reg_ver_win_rate',
  'customer_idx',
  'historical_existing_cnt',
  'id_strategic_ver',
  'it_strategic_ver',
  'idit_strategic_ver',
  'lead_desc_length',
  'prefer_ver_count',
  'prefer_ver_mean',
  'ver_cus',
  'ver_pro',
  'ver_win_rate_x',
  'ver_win_ratio_per_bu',
  'lead_owner']

In [11]:
# 범주형 변수
categorical_vars = [
    'expected_budget', 'bant_submit', 'customer_country', 'business_unit', 'customer_idx', 'customer_type', 
    'enterprise', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job',
    'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 
    'customer_country.1', 'customer_position', 'response_corporate', 'expected_timeline', 
    'ver_cus', 'ver_pro', 'business_area', 'business_subarea', 'lead_owner', 
    'Detailed_Address', 'City', 'Country', 'is_converted', 'customer_history', 'transfer_agreement', 'lead_description',
    'event_split_1', 'event_split_2'
]

# 수치형 변수
numerical_vars = [
    'com_reg_ver_win_rate', 'historical_existing_cnt', 'lead_desc_length','ver_win_rate_x',
    'ver_win_ratio_per_bu', 'ver_pro', 'ver_win_rate_x', 'ver_win_rate_mean_upper'
]

# 모든 유지할 컬럼의 리스트를 만듭니다.
columns_to_keep = categorical_vars + numerical_vars

# drop할 컬럼을 식별합니다. 이는 전체 컬럼에서 columns_to_keep에 없는 컬럼들입니다.
columns_to_drop = [col for col in train_data.columns if col not in columns_to_keep]
columns_to_drop_2 = [col for col in test_data.columns if col not in columns_to_keep]

columns_to_drop
# 불필요한 컬럼을 제거합니다.
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop_2)


In [12]:
# 결측치 비율과 범주형, 수치형 변수 출력
missing_values, categorical_vars, numerical_vars

# train_data에서 결측치가 90% 이상인 컬럼을 찾습니다.
columns_to_drop = train_data.columns[train_data.isnull().mean() > 0.9]

# 찾은 컬럼들을 train_data와 test_data에서 제거합니다.
df_reduced = train_data.drop(columns=columns_to_drop)
test_df_reduced = test_data.drop(columns=columns_to_drop)





# 지역 정보 전처리, 동일한 칼럼 삭제
df_reduced = df_reduced.drop(['customer_country.1'], axis = 1)
test_df_reduced = test_df_reduced.drop(['customer_country.1'], axis = 1)

# 주소 데이터 분리 로직 수정
def extract_address_parts_fixed(address):
    if pd.isna(address) or address.strip() == '':
        return 'Unknown', 'Unknown', 'Unknown'  # 결측치인 경우, 모든 반환값을 None으로 설정

    parts = address.split('/')
    country = parts[-1].strip() if len(parts) > 1 and parts[-1].strip() != '' else 'Unknown'
    city = parts[-2].strip() if len(parts) > 2 and parts[-2].strip() != '' else 'Unknown'
    detailed_address = '/'.join(parts[:-2]).strip() if len(parts) > 2 and '/'.join(parts[:-2]).strip() != '' else 'Unknown'

    # 슬래시로 시작하는 경우, 첫 슬래시 제거
    if detailed_address and detailed_address.startswith('/'):
        detailed_address = detailed_address[1:]

    return detailed_address, city, country

In [13]:
# 주소 분리 함수 적용하여 새로운 컬럼 생성
df_reduced['Detailed_Address'], df_reduced['City'], df_reduced['Country'] = zip(*df_reduced['customer_country'].apply(extract_address_parts_fixed))
test_df_reduced['Detailed_Address'], test_df_reduced['City'], test_df_reduced['Country'] = zip(*test_df_reduced['customer_country'].apply(extract_address_parts_fixed))

# 분리 결과 확인
df_reduced[['customer_country', 'Detailed_Address', 'City', 'Country']].head(10)
test_df_reduced[['customer_country', 'Detailed_Address', 'City', 'Country']].head(10)

# 국가명 표기법 표준화 매핑 규칙 정의
country_standardization_rules = {
    "U.A.E": "United Arab Emirates",
    # 추가적인 국가명 변환 규칙이 필요한 경우, 여기에 추가
}

# 국가명 표기법 표준화 함수 정의
def standardize_country_name(country_name):
    # 매핑 규칙에 따라 국가명 변환
    return country_standardization_rules.get(country_name, country_name)

# 'Country' 컬럼의 국가명 표기법 표준화 적용
df_reduced['Country'] = df_reduced['Country'].apply(standardize_country_name)
test_df_reduced['Country'] = test_df_reduced['Country'].apply(standardize_country_name)

# 표준화된 국가명 컬럼의 고유값 확인을 위한 샘플 출력
standardized_country_samples = df_reduced['Country'].unique()[:10]  # 처음 10개의 고유값 샘플


# 'Country_Corrected' 컬럼의 국가명 대소문자 통일
df_reduced['Country'] = df_reduced['Country'].str.title()
test_df_reduced['Country'] = test_df_reduced['Country'].str.title()

# bant_submit 컬럼을 문자열로 변환
df_reduced['bant_submit'] = df_reduced['bant_submit'].astype(str)
test_df_reduced['bant_submit'] = test_df_reduced['bant_submit'].astype(str)


# id_strategic_ver 컬럼을 문자열로 변환
df_reduced['id_strategic_ver'] = df_reduced['id_strategic_ver'].astype(str)
test_df_reduced['id_strategic_ver'] = test_df_reduced['id_strategic_ver'].astype(str)

# it_strategic_ver 컬럼을 문자열로 변환
df_reduced['it_strategic_ver'] = df_reduced['it_strategic_ver'].astype(str)
test_df_reduced['it_strategic_ver'] = test_df_reduced['it_strategic_ver'].astype(str)

# idit_strategic_ver 컬럼을 문자열로 변환
df_reduced['idit_strategic_ver'] = df_reduced['idit_strategic_ver'].astype(str)
test_df_reduced['idit_strategic_ver'] = test_df_reduced['idit_strategic_ver'].astype(str)

# 범주형 변수의 결측치를 'Unknown'으로 채우기
categorical_vars_with_missing =  [
    'expected_budget', 'bant_submit', 'business_unit', 'customer_idx', 'customer_type', 
    'enterprise', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job',
    'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 
    'customer_position', 'response_corporate', 'expected_timeline', 
    'ver_cus', 'ver_pro', 'business_area', 'lead_owner', 
    'Detailed_Address', 'City', 'Country', 'is_converted', 'customer_history', 'transfer_agreement', 'lead_description',
    'event_split_1', 'event_split_2'
]

for var in categorical_vars_with_missing:
    df_reduced[var].fillna('Unknown', inplace=True)
    test_df_reduced[var].fillna('Unknown', inplace=True)

df_reduced['historical_existing_cnt'].fillna(0, inplace=True)
test_df_reduced['historical_existing_cnt'].fillna(0, inplace=True)

# 결측치 처리 후 데이터 확인
df_reduced[categorical_vars_with_missing].isnull().sum()
test_df_reduced[categorical_vars_with_missing].isnull().sum()

df_reduced = df_reduced.drop(['customer_country'], axis = 1)
test_df_reduced = test_df_reduced.drop(['customer_country'], axis = 1)

In [14]:
# df_reduced 데이터프레임의 'lead_desc_length' 컬럼을 십의 자리에서 내림
df_reduced['lead_desc_length'] = np.floor(df_reduced['lead_desc_length'] / 10) * 10

# test_df_reduced 데이터프레임의 'lead_desc_length' 컬럼을 십의 자리에서 내림
test_df_reduced['lead_desc_length'] = np.floor(test_df_reduced['lead_desc_length'] / 10) * 10

In [15]:
columns_info = []

# 각 컬럼별로 반복
for column in df_reduced.columns:
    # 유니크 값과 그 갯수
    unique_values = df_reduced[column].unique()
    unique_counts = df_reduced[column].nunique()
    
    # 결측치 비율 계산
    missing_ratio = df_reduced[column].isnull().mean() * 100
    
    # 정보 저장
    columns_info.append({
        'Column': column,
        'Unique Values': unique_values,
        'Number of Unique Values': unique_counts,
        'Missing Value Ratio (%)': missing_ratio
    })

# 결과를 DataFrame으로 변환하여 보기 좋게 출력
columns_info_df = pd.DataFrame(columns_info)
columns_info_df

,Column,Unique Values,Number of Unique Values,Missing Value Ratio (%)
0,bant_submit,"[0.75, 0.25, 1.0, 0.5, 0.0]",5,0.000000
1,expected_budget,"[Unknown, $100,000 ~ $500,000, $50,000 ~ $100,000, more than $1,000,000, $200,000 ~ $400,000, less than $10,000, less than $100,000, $10,000 ~ $50,000, ongoing, $500,000 ~ $1,000,000, less than $50,000, 500.000€ ~ 1.000.000€, more than $50,000, menos de 10.000€, $400,000 ~ $600,000, $500,000 to less than $1m, $100,000 ~ $200,000, less than £7,000, €10,000 ~ €50,000, weniger als €10,000, €100,000 ~ €500,000, €50,000 ~ €100,000, $600,000 ~ $800,000, $800,000 ~ $1,000,000, 25000 usd, on~going, more than £700,000, $10m to less than $25m, $50.337.000 cop, more than $1.000.000, 8.000.000 cop, £7,000 ~ £35,000, n/a, über €1,000,000, cop 5.000.000 ~ 9.000.000, $1m to less than $10m, 700, (select id budget), 9.000.000 cop, don't know/not sure, £35,000 ~ £70,000, 10.000€ ~ 50.000€, no know, menos de $10,000, entre 10.000€ y 50.000€, $500.000 ~ $1.000.000, €500,000 ~ €1,000,000, 100000 $, 5000, 30.000 $ ~ 50.000 $, 1000, entre 50.000€ y 100.000€, cop 2.000.000, residential (home), r$100.000 ~ r$500.000, cop 5.000.000, budget, £70,000 ~ £350,000, 10.000.000 ~ 20.000.000, less than idr 155,000,000, 0, € 5000,00, 10.000 $ ~ 50.000 $, cop 3.000.000, more than $123,000, less than $123,000, entre 500.000€ y 1.000.000€, more than $100,000, $50.000 ~ $10.000, cop 15.000.000 ~ 30.000.000]",70,0.000000
2,business_unit,"[AS, ID, IT, Solution, CM]",5,0.000000
3,com_reg_ver_win_rate,"[nan, 0.3902439024390244, 0.0175438596491228, 0.0289256198347107, 0.0446428571428571, 0.0039370078740157, 0.1052631578947368, 0.0732484076433121, 0.0677966101694915, 0.0037878787878787, 0.04, 0.0888888888888888, 0.0199004975124378, 0.0833333333333333, 0.3333333333333333, 0.0151515151515151, 0.004, 0.0118577075098814, 0.0749486652977412, 0.0330578512396694, 0.6428571428571429, 0.0196078431372549, 0.0575342465753424, 0.0538922155688622, 0.0311958405545927, 0.0695652173913043, 0.1241217798594847, 0.0327868852459016, 0.0714285714285714, 0.0491803278688524, 0.0843373493975903, 0.0109890109890109, 0.0485436893203883, 0.075, 0.0681818181818181, 0.0181818181818181, 0.1363636363636363, 0.1470588235294117, 0.037037037037037, 0.0496894409937888, 0.043103448275862, 0.0169491525423728, 0.0869565217391304, 0.0806916426512968, 0.0289855072463768, 0.0227272727272727, 0.036036036036036, 0.0202020202020202, 0.0666666666666666, 0.1184210526315789, 0.032258064516129, 0.0476190476190476, 0.0422535211267605, 0.0135135135135135, 0.1186440677966101, 0.1136363636363636, 0.6153846153846154, 0.0544217687074829, 0.0172413793103448, 0.1162790697674418, 0.4615384615384615, 0.2307692307692307, 0.2692307692307692, 0.0434782608695652, 0.0416666666666666, 0.025, 0.1666666666666666, 0.0531914893617021, 0.0408163265306122, 0.125, 0.0555555555555555, 0.5, 0.2142857142857142, 0.1818181818181818, 0.4444444444444444, 1.0, 0.25, 0.3636363636363636, 0.8333333333333334, 0.4, 0.2]",80,75.432975
4,customer_idx,"[33931, 42490, 10350, 19750, 45802, 6837, 47466, 22031, 24755, 42492, 46230, 22805, 3638, 8155, 5764, 11420, 44226, 12153, 25096, 23800, 1668, 8994, 12809, 39958, 23756, 33763, 26714, 20719, 31223, 33350, 35222, 11186, 33357, 10372, 43811, 43627, 41906, 22135, 44781, 13197, 38997, 37124, 18605, 24472, 42387, 7118, 21863, 23494, 32912, 16496, 41227, 38532, 12830, 34556, 12883, 23520, 20346, 1693, 12095, 23546, 37475, 38106, 38766, 44283, 32240, 21701, 44949, 17319, 46104, 16130, 22102, 123, 42489, 18030, 1482, 11221, 14746, 25309, 31132, 43414, 2328, 40717, 11442, 35004, 30733, 8750, 38985, 9881, 30784, 25570, 2087, 11037, 43116, 32328, 3215, 43296, 45330, 29513, 31509, 34845, ...]",35112,0.000000
5,customer_type,"[End Customer, Unknown, Solution Eco-Partner, Specifier / Influencer, Service Partner, Channel Partner, Etc., Software/Solution Provider, Dealer/Distributor, Installer/Contractor, Other, End-user, Developer, Technician, Corporate, Engineer, Others, Consultan

In [16]:
# Based on the analysis and the identified descriptions with a 100% conversion rate,
# we update the positive and negative keywords lists for the final preprocessing code implementation.

# Implement the final preprocessing function with updated keywords
def final_preprocess_expected_timeline(description):
     # Define positive and negative keywords based on previous analysis
    positive_keywords = [    'request', 'interested', 'demo', 'quote', 'detail',  # Original keywords
    'boq of requirement', 'requirement pending new quote', 'customer wants demo', 'looking for', 'details shared',
    'demo completed', 'demo scheduled', 'shared with him', 'discussed with client', 'require demo',
                             'quote has been sent to customer.', 'up to december']
    negative_keywords = [    'not interested', 'no requirement', 'not responding', 'closed',
        'partner was exploring on eol model for some requirement. no designated sales or closure expected hence closing in the system.',
        'client not interested in product.. receing call and not answering properly',
        'already close in july purchse with rd',
        'quote shared with him.',
        'not responding to calls'
    ]
    other_keywords = ['being followed up','being followed up.' ]
    # Check for NaN values and return 'nan'
    if pd.isna(description):
        return 'nan'
    description_lower = description.lower()

    # Check if the description contains any updated positive keywords
    if any(keyword in description_lower for keyword in positive_keywords):
        return 'positive'
    # Check if the description contains any negative keywords
    elif any(keyword in description_lower for keyword in negative_keywords):
        return 'negative'
    # Predefined time ranges
    elif description_lower in ['less than 3 months', '3 months ~ 6 months', '9 months ~ 1 year', 'more than a year', '6 months ~ 9 months']:
        return description_lower
    elif description_lower in other_keywords:
        return 'other'
    else:
        return 'other'

# Apply the final preprocessing function to the "expected_timeline" column
df_reduced['expected_timeline'] = df_reduced['expected_timeline'].apply(final_preprocess_expected_timeline)
test_df_reduced['expected_timeline'] = test_df_reduced['expected_timeline'].apply(final_preprocess_expected_timeline)

In [17]:
# Define initial positive and negative keywords based on the provided example and common sense
positive_keywords = [
    'request', 'interested', 'demo', 'quote', 'detail',
    'boq of requirement', 'requirement pending new quote', 'customer wants demo', 'looking for', 'details shared',
    'demo completed', 'demo scheduled', 'shared with him', 'discussed with client', 'require demo',
    'quote has been sent to customer.', 'up to december'
]
negative_keywords = [
    'not interested', 'no requirement', 'not responding', 'closed',
    'partner was exploring on eol model for some requirement. no designated sales or closure expected hence closing in the system.',
    'client not interested in product.. receiving call and not answering properly',
    'already close in july purchase with rd',
    'quote shared with him.',
    'not responding to calls','not'
]

# Function to classify descriptions based on the initial keywords
def classify_description(description):
    if pd.isna(description):
        return 'other'  # Using 'other' for missing descriptions for simplicity
    description_lower = description.lower()
    if any(keyword in description_lower for keyword in positive_keywords):
        return 'positive'
    elif any(keyword in description_lower for keyword in negative_keywords):
        return 'negative'
    else:
        return 'other'

# Apply the classification function to the 'lead_description' column
df_reduced['lead_description'] = df_reduced['lead_description'].apply(classify_description)
# Apply the classification function to the 'lead_description' column
test_df_reduced['lead_description'] = test_df_reduced['lead_description'].apply(classify_description)

In [18]:
from sklearn.model_selection import train_test_split

# 타겟 변수와 특성 분리
X = df_reduced.drop('is_converted', axis=1)
y = df_reduced['is_converted']

# 데이터를 훈련 세트와 검증 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
    shuffle=True,
    random_state=400)

# 분할된 데이터 세트의 크기 확인
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((47439, 35), (11860, 35), (47439,), (11860,))

In [19]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  object 
 1   expected_budget          59299 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            59299 non-null  object 
 6   customer_history         59299 non-null  object 
 7   enterprise               59299 non-null  object 
 8   historical_existing_cnt  59299 non-null  float64
 9   id_strategic_ver         59299 non-null  object 
 10  it_strategic_ver         59299 non-null  object 
 11  idit_strategic_ver       59299 non-null  object 
 12  customer_job             59299 non-null  object 
 13  lead_description         59299 non-null  object 
 14  lead_desc_length      

In [20]:
from catboost import CatBoostClassifier

# 범주형 변수 인덱스 식별
cat_features_indices = [X_train.columns.get_loc(c) for c in  categorical_vars if c in X_train]
print(cat_features_indices)

# CatBoost 모델 초기화
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, cat_features=cat_features_indices, verbose=10)

# 모델 학습
model.fit(X_train, y_train)

[1, 0, 2, 4, 5, 7, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 23, 24, 28, 29, 32, 33, 34, 6, 22, 13, 30, 31]
0:	learn: 0.5538394	total: 91.3ms	remaining: 9.04s
10:	learn: 0.1109732	total: 400ms	remaining: 3.24s
20:	learn: 0.0833153	total: 654ms	remaining: 2.46s
30:	learn: 0.0783710	total: 906ms	remaining: 2.02s
40:	learn: 0.0745143	total: 1.16s	remaining: 1.67s
50:	learn: 0.0720951	total: 1.41s	remaining: 1.35s
60:	learn: 0.0701185	total: 1.66s	remaining: 1.06s
70:	learn: 0.0684354	total: 1.91s	remaining: 779ms
80:	learn: 0.0672111	total: 2.16s	remaining: 507ms
90:	learn: 0.0663073	total: 2.41s	remaining: 238ms
99:	learn: 0.0656921	total: 2.64s	remaining: 0us


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 예측 값이 있어야 합니다. 예를 들어: prediction
# 예측 확률 계산
pred_probs = model.predict_proba(X_test)[:, 1]

# 임계값을 다양하게 설정하여 평가
thresholds = [0.3, 0.35, 0.38, 0.4, 0.42, 0.43, 0.44, 0.45, 0.5]
for thresh in thresholds:
    # 임계값에 따른 예측 클래스 결정
    predictions = (pred_probs >= thresh).astype(int)

    # 성능 평가
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)

    print(f"Threshold: {thresh}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Threshold: 0.3, Accuracy: 0.9782462057335581, Precision: 0.8605371900826446, Recall: 0.8713389121338913, F1 Score: 0.8659043659043659
Threshold: 0.35, Accuracy: 0.9794266441821248, Precision: 0.8973214285714286, Recall: 0.8410041841004184, F1 Score: 0.8682505399568035
Threshold: 0.38, Accuracy: 0.9798482293423272, Precision: 0.9125431530494822, Recall: 0.8294979079497908, F1 Score: 0.8690410958904109
Threshold: 0.4, Accuracy: 0.9795109612141653, Precision: 0.9169590643274854, Recall: 0.8200836820083682, F1 Score: 0.8658199889563777
Threshold: 0.42, Accuracy: 0.9796795952782462, Precision: 0.927120669056153, Recall: 0.8117154811715481, F1 Score: 0.8655883993307306
Threshold: 0.43, Accuracy: 0.9794266441821248, Precision: 0.9278846153846154, Recall: 0.8075313807531381, F1 Score: 0.8635346756152126
Threshold: 0.44, Accuracy: 0.9792580101180438, Precision: 0.928743961352657, Recall: 0.8043933054393305, F1 Score: 0.8621076233183856
Threshold: 0.45, Accuracy: 0.9793423271500843, Precision: 0

In [26]:
# 예측 확률 계산
# 주의: 이 코드는 실제 모델과 submission_df에 적용하기 전에 필요한 전처리가 완료되었다고 가정합니다.
submission_df = test_df_reduced.drop(['is_converted'], axis=1)
pred_probs = model.predict_proba(submission_df)[:, 1]

# 임계값 적용하여 최종 예측 결정
threshold = 0.248
predictions = (pred_probs >= threshold).astype(int)  # 양성 클래스를 1, 음성 클래스를 0으로 표현

submission = pd.read_csv("submission.csv", encoding = 'latin1') # 학습용 데이터

# 예측 결과를 submission_df에 추가
submission['is_converted'] = predictions
submission['id'] = submission_id
submission.to_csv("submission.csv", index = False)

sum(submission['is_converted'])

1924